#**OLAP Cube**

In [55]:
pip install cubes

In [56]:
pip install sqlalchemy==1.4.4

In [ ]:
from sqlalchemy import create_engine
from cubes.tutorial.sql import create_table_from_csv
from cubes import Workspace, Cell, browser, PointCut

import configparser

** z from collections import MutableMapping na from collections.abc import MutableMapping upravit py soubory**

In [ ]:
engine = create_engine('sqlite:///data.sqlite')
create_table_from_csv(engine,
                      "data.csv",
                      table_name="irbd_balance",
                      fields=[
                            ("category", "string"),
                            ("category_label", "string"),
                            ("subcategory", "string"),
                            ("subcategory_label", "string"),
                            ("line_item", "string"),
                            ("year", "integer"),
                            ("amount", "integer")],
                      create_id=True
                     )

In [ ]:
x = configparser.ConfigParser()
x.read("slicer.ini")
workspace = Workspace(config=x)
#workspace.register_default_store("sql", url="sqlite:///data.sqlite")
workspace.import_model("model.json")

In [ ]:
browser = workspace.browser("irbd_balance")

In [57]:
result = browser.aggregate()
print("Total\n"
      "----------------------")
print("Record count : %8d" % result.summary["record_count"])
print("Total amount : %8d" % result.summary["amount_sum"])
print("Double amount: %8d" % result.summary["double_amount_sum"])

Total
----------------------
Record count :       62
Total amount :  1116860
Double amount:  2233720


In [58]:
cube = workspace.cube("irbd_balance")
cuts = [
    PointCut("year", [2010])
]
cell = Cell(cube, cuts)
result = browser.aggregate(cell)

print("Total\n"
      "----------------------")
print("Record count : %8d" % result.summary["record_count"])
print("Total amount : %8d" % result.summary["amount_sum"])
print("Double amount: %8d" % result.summary["double_amount_sum"])

Total
----------------------
Record count :       31
Total amount :   566020
Double amount:  1132040


#**Pivot Table**

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("data.csv")
print(df)

   Category Code Category Subcategory Code        Subcategory  \
0              a   Assets              dfb     Due from Banks   
1              a   Assets              dfb     Due from Banks   
2              a   Assets              dfb     Due from Banks   
3              a   Assets              dfb     Due from Banks   
4              a   Assets                i        Investments   
..           ...      ...              ...                ...   
57             e   Equity               da   Deferred Amounts   
58             e   Equity               re  Retained Earnings   
59             e   Equity               re  Retained Earnings   
60             e   Equity               oe              Other   
61             e   Equity               oe              Other   

                                            Line Item  Fiscal Year  \
0                             Unrestricted currencies         2010   
1                             Unrestricted currencies         2009   
2        

In [ ]:
pivot = df.pivot_table(index=['Category'],
                       values=['Amount (US$, Millions)'],
                       aggfunc='sum')
print(pivot)

             Amount (US$, Millions)
Category                           
Assets                       558430
Equity                        77592
Liabilities                  480838


In [ ]:
pivot = df.pivot_table(index=['Subcategory'],
                       values=['Amount (US$, Millions)'],
                       aggfunc='sum')
print(pivot)

                        Amount (US$, Millions)
Subcategory                                   
Borrowings                              238617
Capital Stock                            22983
Deferred Amounts                           672
Derivative Assets                       244691
Derivative Liabilities                  226060
Due from Banks                            4847
Investments                              77024
Loans Outstanding                       221761
Nonnegotiable                             2325
Other                                    -4661
Other Assets                              5318
Other Liabilities                        12775
Other Receivables                         1795
Receivables                                347
Retained Earnings                        58663
Securities                                 322
Sold or Lent                              3321


In [ ]:
pivot = df.pivot_table(index=['Category', 'Subcategory'],
                       values=['Amount (US$, Millions)'], aggfunc='sum')
print(pivot)

                                    Amount (US$, Millions)
Category    Subcategory                                   
Assets      Derivative Assets                       244691
            Due from Banks                            4847
            Investments                              77024
            Loans Outstanding                       221761
            Nonnegotiable                             2325
            Other Assets                              5318
            Other Receivables                         1795
            Receivables                                347
            Securities                                 322
Equity      Capital Stock                            22983
            Deferred Amounts                           672
            Other                                    -4726
            Retained Earnings                        58663
Liabilities Borrowings                              238617
            Derivative Liabilities                  2260

In [ ]:
pivot = df.pivot_table(index=['Subcategory'],
                       values=['Amount (US$, Millions)'],
                       aggfunc={'median', 'mean', 'min'})
print(pivot)

                       Amount (US$, Millions)                  
                                         mean    median     min
Subcategory                                                    
Borrowings                      119308.500000  119308.5  110040
Capital Stock                    11491.500000   11491.5   11491
Deferred Amounts                   336.000000     336.0     313
Derivative Assets                30586.375000   18050.0    2246
Derivative Liabilities           28257.500000   18273.0     780
Due from Banks                    1211.750000    1122.5     222
Investments                      38512.000000   38512.0   36012
Loans Outstanding               110880.500000  110880.5  103657
Nonnegotiable                     1162.500000    1162.5    1123
Other                            -1165.250000    -837.5   -3043
Other Assets                       886.333333     630.0       0
Other Liabilities                 1596.875000    1342.5     307
Other Receivables                  448.7

In [ ]:
pivot = df.pivot_table(index=['Category'],
                       values=['Amount (US$, Millions)'],
                       aggfunc={'median', 'mean', 'min'})
print(pivot)

            Amount (US$, Millions)              
                              mean  median   min
Category                                        
Assets                17450.937500  1249.5     0
Equity                 9699.000000  5925.0 -3043
Liabilities           21856.272727  2390.0     8
